In [8]:
import numpy as np

In [13]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits
import pandas as pd
df = pd.read_csv("C:\\Users\\hp\\Desktop\\faqs.csv",encoding_errors='ignore')
# pd.set_option('display.max_colwidth', None)
df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) \
#                                    .pipe(remove_lessthan,length=3) \
#                                    .pipe(remove_stopwords,stopwords=en_stopwords.union(hi_stopwords))
df

C:\Users\hp\Anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kandee) is a platform that h...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
q_embs = model.encode(df['clean_Q'], convert_to_tensor=True)

C:\Users\hp\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [16]:
q_embs.shape

torch.Size([6, 384])

In [31]:
user_query = 'what do you know about kandi?'

In [32]:
df_user_query = pd.DataFrame([user_query], columns=['user_query'])

In [33]:
df_user_query['clean_user_query'] = df_user_query['user_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation)
df_user_query

C:\Users\hp\Anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,user_query,clean_user_query
0,what do you know about kandi?,what do you know about kandi


In [35]:
user_q_embs = model.encode(df_user_query['clean_user_query'], convert_to_tensor=True)
user_q_embs.shape

torch.Size([1, 384])

In [36]:
import torch

In [37]:
cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [44]:
index = np.argmax(cos_fn(user_q_embs, q_embs)).item()

In [45]:
df['A'][index]

'kandi (pronounced kandee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [ ]:
def pred_ans(user_query):
    df_user_query = pd.DataFrame([user_query], columns=['user_query'])
    df_user_query['clean_user_query'] = df_user_query['user_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation)
    user_q_embs = model.encode(df_user_query['clean_user_query'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    index = np.argmax(cos_fn(user_q_embs, q_embs)).item()
    return df['A'][index]

In [ ]:
while True:
    user_query = input('Ask your query:')
    if user_query == 'exit':
        break
    else:
        print(pred_ans(user_query))
        print('***************')